In [15]:
import random
import time
import pandas as pd
import numpy as np


class Rules:
    def __init__(self, bj_payout=1.5, hit_soft17=True, allow_surrender=True, double_after_split=True, resplit_aces=False, hit_split_aces=False):
        self.bj_payout = bj_payout
        self.hit_soft17 = hit_soft17
        self.allow_surrender = allow_surrender
        self.double_after_split = double_after_split
        self.resplit_aces = resplit_aces
        self.hit_split_aces = hit_split_aces
        
class Counter:
    def __init__(self):
        values = [2,3,4,5,6,7,8,9,10,11,1]
        self.systems = {
#             'hiopt1': dict(zip(values, [0,1,1,1,1,0,0,0,-1,0,0])),
            'hilow': dict(zip(values, [1,1,1,1,1,0,0,0,-1,-1,-1])),
            'tencount': dict(zip(values, [1,1,1,1,1,1,1,1,-2,0,0])),
#             'aces': dict(zip(values, [0,0,0,0,0,0,0,0,0,-1,-1]))
        }
        self.counts = {system: 0 for system in self.systems}
#         self.observed = dict(zip(values, [0,0,0,0,0,0,0,0,0,0]))
        
    def count(self, card):
#         self.observed[card] += 1
        for system in self.systems:
            self.counts[system] += self.systems[system][card]
    
    def copy(self):
        counter = Counter()
        counter.counts = self.counts.copy()
#         counter.observed = self.observed.copy()
        return counter

class Deck:
    def __init__(self, rules, num_decks, shuffle=True):
        self.rules = rules
        self.num_decks = num_decks
        self.cards = [2,3,4,5,6,7,8,9,10,10,10,10,11] * 4 * self.num_decks
        if shuffle:
            for i in range(5):
                random.shuffle(self.cards)
        self.counter = Counter()
            
    def deal_card(self, count=True):
        card = self.cards.pop()
        if count:
            self.counter.count(card)
        return card
    
    def count(self, card):
        self.counter.count(card)
    
    def cards_remaining(self):
        return len(self.cards)
        
    def decks_remaining(self):
        return self.cards_remaining()/52.
    
    def true_count(self):
        return {k: v/self.decks_remaining() for k, v in self.counter.counts.items()}
    
    def copy(self):
        deck = Deck(self.rules, self.num_decks, shuffle=False)
        deck.cards = self.cards[:]
        deck.counter = self.counter.copy()
        return deck
    
class Hand:
    def __init__(self, deck, cards=[], splitted=False, is_dealer=False, actionable=True):
        self.cards = cards[:]
        self.value = 0
        while len(self.cards) < 2:
            if is_dealer and len(self.cards)==0:
                self.cards.append(deck.deal_card(count=False))
            self.cards.append(deck.deal_card())
        
        self.is_dealer = is_dealer
        self.splitted = splitted
        self.actionable = actionable
        self.first_action = len(self.cards)==2
        assert self.first_action
        
        self.value = sum(self.cards)
        if self.value > 21:
            self.pair = self.first_action
            self.cards[self.cards.index(11)] = 1
            self.value -= 10
        else:
            self.pair = self.first_action and self.cards[0] == self.cards[1] 
        
        self.soft = 11 in self.cards
        self.busted = self.value > 21
        self.natural = (self.value == 21) and self.first_action and not self.splitted
        self.surrendered = False
        self.doubled = False
    
    def hit(self, deck, double=False, count=True):
        card = deck.deal_card(count)
        self.cards.append(card)
        self.value += card
        self.first_action = False
        self.pair = False
        self.doubled = double
        self.soft = self.soft or card==11
        while self.value > 21 and self.soft:
            self.cards[self.cards.index(11)] = 1
            self.value -= 10
            self.soft = 11 in self.cards
        if self.value > 21:
            self.busted = True
        return card
    
    def split(self, deck):
        assert self.first_action and self.pair
        act = deck.rules.hit_split_aces or self.to_str()!='AA'
        return Hand(deck, cards=[self.cards[0]], splitted=True, actionable=act), Hand(deck, cards=[self.cards[1]], splitted=True, actionable=act)
    
    def play_hand(self, deck, dealer, action_table, alternate_table=None):
        assert not self.is_dealer
        data = {'player': self.to_str(), 'dealer': dealer.to_str(), 'decks_remaining': deck.decks_remaining()}
        data.update(deck.true_count())
        
        if self.natural or dealer.natural or self.busted or self.value==21 or not self.actionable:
            deck.count(dealer.cards[0])
            data['action'] = None
            data['payout'] = resolve_hand([self], dealer, deck.rules)
            return [data]
        
        alldata = []
        action = action_table.loc[self.to_str(), dealer.to_str()]
        if alternate_table is not None:
            alt = alternate_table.loc[self.to_str(), dealer.to_str()]
#         print(action, alt)
            assert alt=='' or action[0] in alt
            actions = [a for a in alt if a!=action[0]]
        
            for a in actions:
                if a == 'R':
                    assert self.first_action and deck.rules.allow_surrender and not self.splitted
                if a == 'P':
                    assert self.pair and deck.rules.double_after_split
                if a == 'D':
                    assert self.first_action
                data_copy = data.copy()
                deck_copy = deck.copy()
                player_copy = self.copy()
                dealer_copy = dealer.copy()

                player_hands = player_copy.play_player(deck_copy, dealer_copy, action_table, action=a)
                if not all(hand.busted for hand in player_hands):
                    dealer_copy.play_dealer(deck_copy)

                data_copy['action'] = a
                data_copy['payout'] = resolve_hand(player_hands, dealer_copy, deck_copy.rules)
                alldata.append(data_copy)
        
        player_hands = self.play_player(deck, dealer, action_table)
        if not all(hand.busted for hand in player_hands):
            dealer.play_dealer(deck)
        data['action'] = action
        data['payout'] = resolve_hand(player_hands, dealer, deck.rules)
        data['dealer_bust'] = dealer.busted
        alldata.append(data)
        return alldata
    
    def play_player(self, deck, dealer, action_table, action=None):
        if self.natural or dealer.natural or self.busted or self.value==21 or not (self.actionable or (self.to_str()=='AA' and deck.rules.resplit_aces)):
            return [self]
        if action is None:
            action = action_table.loc[self.to_str(), dealer.to_str()]
        
        if action[0] == 'R':
            if self.first_action and deck.rules.allow_surrender and not self.splitted:
                self.surrendered = True
                return [self]
            else: action = action[1].upper()
        if action[0] == 'P':
            assert self.pair
            if len(action)==1 or deck.rules.double_after_split:
                if (self.to_str() == 'AA' and self.splitted and not deck.rules.resplit_aces):
                    action = 'H'
                else:
                    hand1, hand2 = self.split(deck)
                    h1 = hand1.play_player(deck, dealer, action_table)
                    h2 = hand2.play_player(deck, dealer, action_table)
                    return h1 + h2
            else: action = action[1].upper()
        if action[0] == 'D':
            if self.first_action and (deck.rules.double_after_split or not self.splitted):
                card = self.hit(deck, double=True)
                return [self]
            elif len(action)==1:
                action = 'H'
            else: action = action[1].upper()
        if action == 'H':
            card = self.hit(deck)
            return self.play_player(deck, dealer, action_table)
        elif action == 'S':
            return [self]
        
        raise Exception
    
    def play_dealer(self, deck):
        assert self.is_dealer
        deck.count(self.cards[0])
        while self.value < 17 or (deck.rules.hit_soft17 and self.value==17 and self.soft):
            card = self.hit(deck)
        return self
    
    def copy(self):
        assert len(self.cards)>=2
        return Hand(None, self.cards, splitted=self.splitted, is_dealer=self.is_dealer, actionable=self.actionable)
        
    def card2str(self, card):
        return {11:'A', 10:'T'}.get(card, str(card))
        
    def to_str(self):
        if self.is_dealer:
            return self.card2str(self.cards[1])
        if self.value == 21:
            return '21'
        if self.pair and self.first_action:
            return self.card2str(self.cards[0])*2
        if self.soft:
            return 'A' + str(self.value-11)
        return str(self.value)
        
def resolve_hand(player_hands, dealer, rules):
    total = 0
    for hand in player_hands:
        payout = 0
        if hand.natural:
            payout = rules.bj_payout if not dealer.natural else 0
        elif dealer.natural:
            payout = -1
        elif hand.surrendered:
            payout = -.5
        elif hand.busted:
            payout = -2 if hand.doubled else -1
        elif dealer.busted:
            payout = 2 if hand.doubled else 1
        elif hand.value > dealer.value:
            payout = 2 if hand.doubled else 1
        elif hand.value < dealer.value:
            payout = -2 if hand.doubled else -1
        total += payout
    return total

class Blackjack:
    def __init__(self, bj_payout=1.5, hit_soft17=True, allow_surrender=False, double_after_split=False, resplit_aces=False, hit_split_aces=False):
        self.rules = Rules(bj_payout, hit_soft17, allow_surrender, double_after_split, resplit_aces, hit_split_aces)
        self.filename = '/Users/alex/Documents/Gambling/Blackjack/action_table_{}deck_{}17.csv'
        self.alternate_table = None #pd.read_csv('/Users/alex/Documents/Gambling/Blackjack/alternate_action_table.csv', index_col=0).fillna('')
        
    # play a single deck of cards and record data on each hand
    def play_single_deck(self, num_hands=1):
        num_decks = 1
        self.deck = Deck(self.rules, num_decks)
        self.action_table = pd.read_csv(self.filename.format(num_decks, 'SH'[self.rules.hit_soft17]), index_col=0)
        self.data = []
        for i in range(6-num_hands):
            payout = self.play_hand(self.deck)
            self.data += payout
        return self.data
            
    def play_double_deck(self, penetration=1.25):
        num_decks = 2
        self.deck = Deck(self.rules, num_decks)
        self.action_table = pd.read_csv(self.filename.format(num_decks, 'SH'[self.rules.hit_soft17]), index_col=0)
        self.data = []
        while self.deck.decks_remaining() > self.deck.num_decks - penetration:
            payout = self.play_hand(self.deck)
            self.data += payout
        return self.data
        
    # play a hand out of the deck, modify data with results, return deck with cards removed
    def play_hand(self, deck, split_hands=1, player=None, dealer=None):
        count = deck.true_count()
        player = Hand(deck)
        dealer = Hand(deck, is_dealer=True)
        
#         alt = self.alternate_table.loc[player.to_str(), dealer.to_str()]
#         player_hands = player.play_player(deck, dealer, self.action_table)
#         dealer = dealer.play_dealer(deck)
        
        data = player.play_hand(deck, dealer, self.action_table, self.alternate_table)
        [d.update({f'{k}_bet': v for k, v in count.items()}) for d in data]
        return data


In [16]:
game = Blackjack(bj_payout=1.5, hit_soft17=True, allow_surrender=False, double_after_split=True, resplit_aces=False, hit_split_aces=False)
payouts = []
i = 1
t = time.time()
while i <= 1:
    data = game.play_double_deck()
    payouts += data
    if len(payouts) > 1e7:
        pd.DataFrame(payouts).to_csv(f'/Users/alex/Documents/Gambling/Blackjack/2deck_3to2_H17_tencount_{i}.csv')
        payouts = []
        i += 1
        print(i)
# sum(payouts), len(payouts), sum(payouts)/len(payouts), (sum([p**2 for p in payouts])**.5/len(payouts))
print(time.time() - t)

KeyboardInterrupt: 

In [19]:
pd.DataFrame(payouts).to_csv(f'/Users/alex/Documents/Gambling/Blackjack/2deck_3to2_H17_tencount_{i}.csv')

In [18]:
len(payouts)

3080931

In [3]:
df = []
for i in range(1,11):
    df.append(pd.read_csv(f'/Users/alex/Downloads/2deck_3to2_S17_surrender_doubleaftersplit_resplitaces_{i}.csv', index_col=0))
df = pd.concat(df)
df['hiopt1_true'] = df['hiopt1']/df['decks_remaining']
df['hilow_true'] = df['hilow']/df['decks_remaining']

/Users/alex/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
mask = (df['decks_remaining'] == df['decks_remaining'].shift(1)) | (df['decks_remaining'] == df['decks_remaining'].shift(-1))
fltrd = df.loc[mask]
fltrd

,player,dealer,decks_remaining,hiopt1,hilow,aces,action,payout,hiopt1_true,hilow_true
9,14,T,0.826923,5,9,-3,R,-0.5,6.046512,10.883721
10,14,T,0.826923,5,9,-3,S,1.0,6.046512,10.883721
11,14,T,0.826923,5,9,-3,H,-1.0,6.046512,10.883721
18,16,9,1.403846,-2,-1,-1,R,-0.5,-1.424658,-0.712329
19,16,9,1.403846,-2,-1,-1,S,-1.0,-1.424658,-0.712329
...,...,...,...,...,...,...,...,...,...,...
10000007,16,T,1.000000,1,1,-6,S,-1.0,1.000000,1.000000
10000008,16,T,1.000000,1,1,-6,Rh,-0.5,1.000000,1.000000
10000011,15,T,0.711538,1,2,-6,H,-1.0,1.405405,2.810811
10000012,15,T,0.711538,1,2,-6,S,-1.0,1.405405,2.810811


In [77]:
fltrd.to_csv('/Users/alex/Downloads/2deck_3to2_S17_surrender_doubleaftersplit_resplitaces_alternate_actions.csv', index=False)